In [1]:
##################################
#
# Initial implementation of linear logic recurrent neural network
#
# The architecture is a modified RNN, see the paper "Linear logic and recurrent neural networks".
# Our inputs are sequences of symbols taken from an alphabet of size num_classes. The length
# of the sequences is N. Our outputs are also sequences of length N from the same alphabet.
#
# An input symbol is encoded as a one_hot vector, so our inputs are sequences of one-hot vectors
# of size num_classes. Put another way, this is the dimension of the input space.
#

# The next three lines are recommend by TF
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import collections
import six
import math
import time

from random import shuffle
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

# Our controller code
import ntm

In [2]:
# GLOBAL FLAGS
num_classes = 2
batch_size = 1000
input_size = num_classes # dimension of the input space I
N = 15 # length of sequences
training_percent = 0.05 # precentage used for training
epoch = 10

controller_state_size = 10 # dimension of the internal state space of the controller
memory_address_size = 5 # number of memory locations
memory_content_size = 5 # size of vector stored at a memory location
state_size = controller_state_size + 2*memory_address_size \
			+ memory_address_size * memory_content_size

def f_ident(seq):
    return seq

func_to_learn = f_ident

In [3]:
# Create a shuffled list of all binary sequences of length N
s = '{0:0' + str(N) + 'b}'
seq_input = [s.format(i) for i in range(2**N)]
shuffle(seq_input)
seq_input = [map(int,i) for i in seq_input]
ti = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(j)
    ti.append(temp_list)
seq_input = ti

print("Number of sequences: " + str(len(seq_input)))
print(seq_input[0])
# A typical element of seq_input at this point will be an array like
# array([[1],[0],[1],[1],[0]])

one_hots = []
for i in range(num_classes):
    a = [0.0]*num_classes
    a[i] = 1.0
    one_hots.append(np.array(a))

seq_input_onehot = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_input_onehot.append(np.array(temp_list))

print(seq_input_onehot[0])

Number of sequences: 32768
[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0]
[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [4]:
# Training output
seq_output = []

# Swaps 0s for 1s in each sequence
for i in seq_input:
    seq_output.append(func_to_learn(i))
                    
print(seq_output[0])

seq_output_onehot = []
for i in seq_output:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_output_onehot.append(np.array(temp_list))

print(seq_output_onehot[0])

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0]
[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [5]:
NUM_EXAMPLES = int(training_percent * len(seq_input))
print("Number of training examples: " + str(NUM_EXAMPLES) + "/" + str(len(seq_input)))

test_input = seq_input_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
test_output = seq_output_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
train_input = seq_input_onehot[:NUM_EXAMPLES]
train_output = seq_output_onehot[:NUM_EXAMPLES]

print("")
print("The first one-hot encoded digit of the first three output sequences")
print(test_output[0][0])
print(test_output[1][0])
print(test_output[2][0])

Number of training examples: 6553/32768

The first one-hot encoded digit of the first three output sequences
[ 1.  0.]
[ 1.  0.]
[ 0.  1.]


In [6]:
# Definition of the model

# inputs, we create N of them, each of shape [None,input_size], one for
# each position in the sequence
inputs = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]
targets = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]

# We use tf.nn.rnn rather than dynamic_rnn because there appears to
# be a problem with tf.map_fn and the latter, at least in 0.10
# state_size is the number of hidden neurons in each layer
cell = ntm.NTMRNNCell(state_size,input_size,controller_state_size,memory_address_size,memory_content_size)

state = cell.zero_state(batch_size, tf.float32)

rnn_outputs = []
reuse = False

for i in range(N):
    output, state = cell(inputs[i],state,'NTM',reuse)
    rnn_outputs.append(output)
    reuse = True

# Final fully connected layer
E = tf.Variable(tf.truncated_normal([controller_state_size,input_size]))
F = tf.Variable(tf.constant(0.1, shape=[input_size]))

# prediction is a length N list of tensors of shape [None,input_size], where
# the jth row of prediction[d] is, for the jth input sequence in the batch,
# the probability distribution over symbols for the output symbol in position d.
logits = [tf.matmul(rnn_output, E) + F for rnn_output in rnn_outputs]
prediction = [tf.nn.softmax(logit) for logit in logits] 
print("prediction shape: " + str(prediction[0].get_shape()))
ce = [tf.reduce_sum(targets[i] * tf.log(prediction[i])) for i in range(N)]

cross_entropy = -tf.add_n(ce)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = [tf.not_equal(tf.argmax(targets[i], 1), tf.argmax(prediction[i], 1)) for i in range(N)]
errors = [tf.reduce_mean(tf.cast(m, tf.float32)) for m in mistakes]

prediction shape: (1000, 2)
[<tf.Tensor 'gradients/NTM_14/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_13/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_12/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_11/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_10/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_9/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_8/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_7/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_6/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_5/split_grad/concat:0' shape=(1000, 45) dtype=float32>, None, None]
[<tf.Tensor 'grad

In [7]:
# Initialise the model
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [8]:
# Display the errors before training
feed_dict = {}
test_input_batch = test_input[:batch_size]
test_output_batch = test_output[:batch_size]
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input_batch)):
        ti.append(test_input_batch[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output_batch[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
print(sess.run(tf.argmax(targets[0],1),feed_dict))
print(sess.run(tf.argmax(prediction[0],1),feed_dict))
print(sess.run(tf.not_equal(tf.argmax(targets[0], 1), tf.argmax(prediction[0], 1)),feed_dict))

print("")
print("The mean of the errors in each digit for the test set:")
incorrects = sess.run(errors, feed_dict)
print(incorrects)
print("Mean: " + str(np.mean(incorrects)))

[0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 1 0 1
 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0
 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0
 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0
 1 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1
 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1
 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1
 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0
 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1
 0 0 0 1 1 1 1 0 1 1 0 0 

In [9]:
pre_train_time = time.time()

# Training
no_of_batches = int(len(train_input)/batch_size)
print("Number of batches: " + str(no_of_batches))

# An annoying thing here is that we cannot use a list as a key in a 
# dictionary. The workaround we found on StackOverflow here:
# http://stackoverflow.com/questions/33684657/issue-feeding-a-list-into-feed-dict-in-tensorflow)

# epoch is a global var
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp = train_input[ptr:ptr+batch_size]
        out = train_output[ptr:ptr+batch_size]
        ptr += batch_size
        
        feed_dict = {}
        for d in range(N):
            in_node = inputs[d]
            out_node = targets[d]
            
            # inp has dimensions [batch_size, N, num_classes] and we want to extract
            # the 2D Tensor of shape [batch_size, num_classes] obtained by setting the
            # second coordinate to d
            ti = []
            to = []
            for k in range(batch_size):
                ti.append(inp[k][d])
                to.append(out[k][d])

            feed_dict[in_node] = np.array(ti)
            feed_dict[out_node] = np.array(to)
            
        sess.run(minimize, feed_dict)
    print("Epoch - " + str(i+1))
    
print("It took", time.time() - pre_train_time, "seconds to train.")

Number of batches: 6
Epoch - 1
Epoch - 2
Epoch - 3
Epoch - 4
Epoch - 5
Epoch - 6
Epoch - 7
Epoch - 8
Epoch - 9
Epoch - 10
It took 3.13616800308 seconds to train.


In [10]:
# Display the errors after training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input_batch)):
        ti.append(test_input_batch[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output_batch[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
data = sess.run([tf.argmax(targets[0],1),
                 tf.argmax(prediction[0],1),
                 cross_entropy],feed_dict)

print("First digits of test outputs (actual)")
print(data[0])
print("First digits of test outputs (predicted)")
print(data[1])
print("Cross-entropy: " + str(data[2]))

# print the mean of the errors in each digit for the test set.
incorrects = sess.run(errors, feed_dict)
print(incorrects)
print("Mean: " + str(np.mean(incorrects)))
sess.close()

First digits of test outputs (actual)
[0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 1 0 1
 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0
 0 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0
 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0
 1 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1
 0 1 0 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 1
 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1
 1 1 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1
 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 1
 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0
 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0